In [1]:
import numpy as np
import import_ipynb
from all_utility_functions import *

importing Jupyter notebook from all_utility_functions.ipynb
Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [2]:
test_passages = np.load("processed_data/test_passage_list.npy")

In [3]:
test_embeddings = []
for i,passage in enumerate(test_passages):
    #print("processing doc", i)
    embeddings = get_bert_embeddings(str(passage))
    test_embeddings.append(embeddings)

In [4]:
test_coref_list = np.load("processed_data/test_coref_list.npy")

In [5]:
test_coref_list[3]

{27: [(63, 64), (193, 194)],
 88: [(66, 67), (109, 110)],
 65: [(93, 94), (178, 179)],
 115: [(96, 100), (140, 144)],
 48: [(102, 104), (136, 138)],
 62: [(115, 119), (162, 166)],
 10: [(215, 215), (220, 220)]}

In [6]:
test_mention_list = []
for doc_coref in test_coref_list:
    mention_list = []
    for key, value in doc_coref.items():
        mention_list += value
    mention_list = sorted(mention_list, key = lambda x: x[0]) 
    test_mention_list.append(mention_list)

In [7]:
len(test_mention_list[0])

15

In [8]:
test_edge_pairs = []
test_samples_input = []

In [9]:
for doc_no, mention_list in enumerate(test_mention_list):
    doc_samples = []
    doc_edges = []
    for i in range(0,len(mention_list)):
        for j in range(i+1, len(mention_list)):
            sample = []
            sample.append(doc_no)
            sample.append(mention_list[i][0])
            sample.append(mention_list[i][1])
            sample.append(mention_list[j][0])
            sample.append(mention_list[j][1])
            doc_samples.append(sample)
            doc_edges.append((i, j))
    test_samples_input.append(doc_samples)
    test_edge_pairs.append(doc_edges)

In [10]:
X_test = test_samples_input[3]

In [11]:
model = Net()
model.load_state_dict(torch.load("processed_data/second_run.pth"))
model.to(device)
model.eval()

Net(
  (fc1): Linear(in_features=2304, out_features=512, bias=True)
  (relu1): ReLU()
  (dout): Dropout(p=0.4)
  (fc2): Linear(in_features=512, out_features=64, bias=True)
  (relu2): ReLU()
  (out): Linear(in_features=64, out_features=1, bias=True)
  (out_act): Sigmoid()
)

In [12]:
X_test = np.array(X_test)
y_preds, y_hats = predict(model, X_test, test_embeddings, 32)

In [13]:
res = (y_preds==1)

In [14]:
pos_samples = torch.nonzero(res.view(-1).data).squeeze().cpu().numpy()
y_probs = y_hats.view(-1).data.squeeze().cpu().numpy()

In [15]:
len(pos_samples)

27

In [16]:
for ind in pos_samples:
    edge = (test_edge_pairs[3][ind])
    print(test_mention_list[3][edge[0]], test_mention_list[3][edge[1]], y_probs[ind])

(63, 64) (66, 67) 1.0
(63, 64) (93, 94) 1.0
(63, 64) (102, 104) 1.0
(63, 64) (109, 110) 1.0
(63, 64) (162, 166) 1.0
(63, 64) (178, 179) 1.0
(63, 64) (193, 194) 1.0
(66, 67) (93, 94) 1.0
(66, 67) (96, 100) 1.0
(66, 67) (102, 104) 1.0
(66, 67) (109, 110) 1.0
(66, 67) (115, 119) 1.0
(66, 67) (136, 138) 1.0
(66, 67) (140, 144) 1.0
(66, 67) (162, 166) 1.0
(66, 67) (178, 179) 1.0
(66, 67) (193, 194) 1.0
(93, 94) (102, 104) 1.0
(93, 94) (109, 110) 1.0
(93, 94) (178, 179) 1.0
(93, 94) (193, 194) 1.0
(109, 110) (178, 179) 1.0
(109, 110) (193, 194) 1.0
(115, 119) (140, 144) 1.0
(115, 119) (162, 166) 1.0
(140, 144) (162, 166) 1.0
(178, 179) (193, 194) 1.0


In [17]:
test_coref_list[3]

{27: [(63, 64), (193, 194)],
 88: [(66, 67), (109, 110)],
 65: [(93, 94), (178, 179)],
 115: [(96, 100), (140, 144)],
 48: [(102, 104), (136, 138)],
 62: [(115, 119), (162, 166)],
 10: [(215, 215), (220, 220)]}